In [2]:
from datetime import datetime
print("This notebook is to test and demonstrate the spectral line fitting and file IO.\n"
	  "This version started 120324, last automatic datestamp:",datetime.now())

This notebook is to test and demonstrate the spectral line fitting and file IO.
This version started 120324, last automatic datestamp: 2025-09-19 14:06:49.235980


In [3]:

import importlib, time, copy, os, sys, numpy as np, astropy.units as u
np.set_printoptions(linewidth=160)

from astropy.io import fits; import matplotlib.pyplot as plt; from matplotlib.gridspec import GridSpec
plt.rcParams.update({'font.size': 18,'figure.figsize':[15,12],'image.origin':'lower'}) # Make the fonts big enough for papers

base_path = '../'
sys.path.append(base_path+'linefit_modules/')
from util import get_mask_errs, get_spice_err
from skew_correction import skew_correct, deskew_linefit_window
from skew_parameter_search import search_shifts, shift_holder, refine_points
from linefit_leastsquares import lsq_fitter, lsq_fitter
from linefit_storage import linefits

fitter = lsq_fitter # lsq_fitter
from linefit_leastsquares import check_for_waves


In [4]:
linelist = {'Ar VIII+S III 700':700.3, 'O III 703':702.9, 'O III 704':703.9, 'Mg IX 706':706.0,
			'O II 718':718.5, 'S IV 745':744.9, 'S IV 748':748.4, 'S IV 750':750.2,
			'O V 759':758.7, 'S IV+O V 759':759.4, 'O V 760':760.3, 'O V 762':762.0,
			'N IV 765':765.1, 'Ne VIII 770':770.4, 'Mg VIII 772':772.3, 'Ne VIII 780':780.3,
			'S V 786':786.5, 'O IV 787':787.7, 'O IV 790':790.1, 'Ly Gamma 972':972.5,
			'C III 977':977.0, 'O I +- Na VI 989':988.7, 'N III 990':989.8, 'N III 992':991.6,
			'H I (+ O I) 1025':1025.7, 'O I 1027':1027.4, 'O VI 1032':1031.9, 'C II 1036':1036.5,
			'O VI 1037':1037.6}

line_names = list(linelist.keys())
line_waves = [linelist[name] for name in line_names]

In [10]:
#base_dir = '/sol/d1/jplowman/solar-physics/'
#sprouts_dir = os.path.join(base_dir,'data','spice','sprouts')
files = np.array(os.listdir('../data'))
fitsfiles = files[[name.endswith('.fits') for name in files]]

win_name = 'O II 718 - Peak (spectral bin 2)' # 'Ly-gamma-CIII group (Merged)' #'O III 703 / Mg IX 706 (Merged)'


## Initial search pattern. Avoid checking 0 shift in either direction because that results in a
## sharper image and therefore more variance:
xl, xh, yl, yh = [-5,5,-5,5]

xs_initial, ys_initial = np.array(np.meshgrid([-5,-2.5,0,2.5,5],[-5,-2.5,0,2.5,5])).transpose([0,2,1])
print(xs_initial, ys_initial)

[[-5.  -5.  -5.  -5.  -5. ]
 [-2.5 -2.5 -2.5 -2.5 -2.5]
 [ 0.   0.   0.   0.   0. ]
 [ 2.5  2.5  2.5  2.5  2.5]
 [ 5.   5.   5.   5.   5. ]] [[-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]]


In [6]:
np.linspace(xl,xh,5)

array([-5. , -2.5,  0. ,  2.5,  5. ])

In [7]:
xs_initial, ys_initial = np.array(np.meshgrid(np.linspace(xl,xh,5),np.linspace(yl,yh,5))).transpose([0,2,1])
print(xs_initial, ys_initial)

[[-5.  -5.  -5.  -5.  -5. ]
 [-2.5 -2.5 -2.5 -2.5 -2.5]
 [ 0.   0.   0.   0.   0. ]
 [ 2.5  2.5  2.5  2.5  2.5]
 [ 5.   5.   5.   5.   5. ]] [[-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]
 [-5.  -2.5  0.   2.5  5. ]]


In [8]:
file_offsets=[0]

In [11]:
hdul = fits.open(os.path.join('../data',fitsfiles[0]))
hdul.info()
hdul.close()

Filename: ../data\solo_L2_spice-n-ras_20220307T030536_V22_100663723-000.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Mg IX 706 - Peak    1 PrimaryHDU     418   (128, 834, 50, 1)   float32   
  1  O II 718 - Peak    1 ImageHDU       417   (128, 834, 50, 1)   float32   
  2  Ne VIII 770 / Mg VIII 772 (Merged)    1 ImageHDU       424   (128, 834, 66, 1)   float32   
  3  S V 786 / O IV 787 - Extend    1 ImageHDU       426   (128, 834, 58, 1)   float32   
  4  Ly Gamma 972 - Peak    1 ImageHDU       417   (128, 834, 36, 1)   float32   
  5  C III 977 (Merged)    1 ImageHDU       426   (128, 834, 42, 1)   float32   
  6  O VI 1032 - Peak    1 ImageHDU       417   (128, 834, 34, 1)   float32   
  7  VARIABLE_KEYWORDS    1 BinTableHDU    398   1R x 28C   [128D, 128I, 128I, 128I, 128I, 128I, 128E, 128E, 128E, 128E, 4I, 4I, 4I, 4I, 4J, 4J, 4J, 4J, 2944A, 128D, 128D, 50D, 50D, 66D, 58D, 36D, 42D, 34D]   
  8  WCSDVARR      1 ImageHDU        28   (128,)   float64   
  9

In [12]:
for i in range(0,len(file_offsets)):
    hdul = fits.open(os.path.join('../data',fitsfiles[file_offsets[i]]))
    spice_dat, spice_hdr = hdul[win_name].data[0], hdul[win_name].header
    spice_dat = spice_dat.transpose([2,1,0]).astype(np.float32)
    hdul.close()
    
    spice_dx, spice_dy, spice_dl = spice_hdr['CDELT1'],spice_hdr['CDELT2'],10*spice_hdr['CDELT3']
    spice_wl0 = 10*spice_hdr['CRVAL3']-spice_dl*spice_hdr['CRPIX3']
    spice_la = spice_wl0+spice_dl*np.arange(spice_dat.shape[2],dtype=np.float64)
    
    centers, lines = check_for_waves(line_waves, line_names, spice_la)
    
    print(lines)
    
    shift_vars = shift_holder(spice_dat, spice_hdr, line_waves, line_names, fitter.__name__, save_dir='../sprout_shift_vars/save/')
    
    sv_initial = search_shifts(spice_dat, spice_hdr, xs_initial, ys_initial, line_waves, line_names, lsq_fitter, single_thread=True, nthreads=32, yrange_plot_dir='../sprout_shift_vars/yrange_plots/', shift_vars=shift_vars)
    
    shift_vars.set(sv_initial)
    
    shift_vars.save()
    
    x_refine, y_refine = refine_points(shift_vars,[-5,5],[-5,5], 11, 11, 20)
    print(x_refine); print(y_refine)
    
    shift_vars = search_shifts(spice_dat, spice_hdr, x_refine, y_refine, line_waves, line_names, lsq_fitter, shift_vars=shift_vars, single_thread=True, nthreads=32, yrange_plot_dir='../sprout_shift_vars/yrange_plots/')
    shift_vars.save()
    
    x_refine, y_refine = refine_points(shift_vars,[-5,5],[-5,5], 31, 31, 20)
    print(x_refine); print(y_refine)
    
    shift_vars = search_shifts(spice_dat, spice_hdr, x_refine, y_refine, line_waves, line_names, lsq_fitter, shift_vars=shift_vars, single_thread=True, nthreads=32, yrange_plot_dir='../sprout_shift_vars/yrange_plots/')
    shift_vars.save()
    
    # Reinterpolate the search results to a finer linear grid for ease of plotting:
    from scipy.interpolate import RegularGridInterpolator, LinearNDInterpolator as lndi
    
    xa = np.array(list(shift_vars.valdict.values()))[:,0]
    ya = np.array(list(shift_vars.valdict.values()))[:,1]
    dat = np.array(list(shift_vars.valdict.values()))[:,2]
    
    include = (np.abs(xa) > 1.0e-5)*(np.abs(ya) > 1.0e-5)
    
    nx_plot, ny_plot = 41, 41
    xya = np.vstack([xa[include],ya[include]]).T
    xa0,ya0 = np.array(np.meshgrid(np.linspace(xl,xh,nx_plot),np.linspace(yl,yh,ny_plot))).transpose([0,2,1])
    dat_interp = lndi(xya, dat[include])(xa0,ya0)
    
    dat_interp = lndi(xya, dat[include])(xa0,ya0)
    
    sort_interp = np.argsort(dat_interp.flatten())
    xsort_interp = xa0.flatten()[sort_interp]
    ysort_interp = ya0.flatten()[sort_interp]
    
    xl2, xh2 = xl-0.5*(xh-xl)/(nx_plot-1), xh+0.5*(xh-xl)/(nx_plot-1)
    yl2, yh2 = yl-0.5*(yh-yl)/(ny_plot-1), yh+0.5*(yh-yl)/(ny_plot-1)
    
    labelstr = spice_hdr['DATE-OBS']+' '+spice_hdr['EXTNAME']
    labelstr = labelstr.replace('-','').replace(':','').replace('  ','_').replace(' ','_')
    labelstr = labelstr.replace('/','_')
    print(labelstr)
    
    fig,axes = plt.subplots(nrows=1,ncols=2,figsize=[16,9])
    plt.suptitle(spice_hdr['DATE-OBS']+' '+spice_hdr['EXTNAME']+': xyshift='+str(np.array([xa[np.argmin(dat)], ya[np.argmin(dat)]])))
    axes[0].imshow(np.clip(np.nansum(spice_dat,axis=2).T,0,None)[150:850,:]**0.5,vmin=0,vmax=(100*np.nanmean(spice_dat))**0.5,aspect=spice_hdr['CDELT2']/spice_hdr['CDELT1'])
    axes[0].set(title='Spectral sum',xlabel='Raster axis @ ypix equivalent -- '+str(spice_hdr['cdelt2'])+'"')
    asdfa = axes[1].imshow(dat_interp.T, extent=[xl2, xh2, yl2, yh2],cmap=plt.get_cmap('gray'))
    axes[1].plot(xa,ya,'P',markersize=10,linewidth=5)
    axes[1].set(title='RMS Doppler variance', xlabel='x shift (arcsecond/angstrom)', ylabel='y shift (arcsecond/angstrom)')
    axes[1].legend(['Sampled points'])
    fig.colorbar(asdfa, ax=axes[1],location='bottom')
    plt.savefig(os.path.join('../sprout_shift_vars/figs','varplot_'+labelstr+'.png'))
    plt.close()

KeyError: "Extension 'O II 718 - Peak (spectral bin 2)' not found."